In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek
from sklearn.calibration import CalibratedClassifierCV
import warnings
warnings.filterwarnings('ignore')

In [ ]:
X = pd.read_csv("../merge/merged_x.csv")
y = pd.read_csv("../merge/merged_y.csv")

In [3]:


def identify_feature_types(df, categorical_threshold=10):
    
    numerical_features = []
    categorical_features = []
    
    for col in df.columns:
        # Agar int yoki float bo'lib, unique values ko'p bo'lsa -> numerical
        if df[col].dtype in ['int64', 'float64']:
            n_unique = df[col].nunique()
            if n_unique > categorical_threshold:
                numerical_features.append(col)
            else:
                categorical_features.append(col)
        else:
            categorical_features.append(col)
    
    return {
        'numerical': numerical_features,
        'categorical': categorical_features
    }

def engineer_features(df, feature_types=None):
    df_enh = df.copy()
    
    # Convert to DataFrame if needed
    if not isinstance(df_enh, pd.DataFrame):
        df_enh = pd.DataFrame(df_enh, columns=[f'feature_{i}' for i in range(df_enh.shape[1])])
    
    # Feature types'ni aniqlash
    if feature_types is None:
        print("🔍 Feature types avtomatik aniqlanmoqda...")
        feature_types = identify_feature_types(df_enh)
    
    numerical_cols = feature_types['numerical']
    categorical_cols = feature_types['categorical']
    
    print(f"\n📊 Feature Types:")
    print(f"   Numerical:   {len(numerical_cols)} features")
    print(f"   Categorical: {len(categorical_cols)} features")
    print(f"\n   Numerical columns: {numerical_cols[:5]}{'...' if len(numerical_cols) > 5 else ''}")
    print(f"   Categorical columns: {categorical_cols[:5]}{'...' if len(categorical_cols) > 5 else ''}")
    
    new_features = []
    
    print("\n🔨 1. Polynomial features (faqat numerical)...")
    
    for col in numerical_cols[:min(5, len(numerical_cols))]:
        try:
            # Squared
            df_enh[f'{col}_squared'] = df_enh[col] ** 2
            new_features.append(f'{col}_squared')
            
            # Cubed (agar variability yuqori bo'lsa)
            if df_enh[col].std() > 0.1:
                df_enh[f'{col}_cubed'] = df_enh[col] ** 3
                new_features.append(f'{col}_cubed')
            
            # Square root (faqat positive values uchun)
            if (df_enh[col] >= 0).all():
                df_enh[f'{col}_sqrt'] = np.sqrt(df_enh[col])
                new_features.append(f'{col}_sqrt')
            
            # Log transform (faqat positive values uchun)
            if (df_enh[col] > 0).all():
                df_enh[f'{col}_log'] = np.log1p(df_enh[col])
                new_features.append(f'{col}_log')
        except Exception as e:
            print(f"   ⚠️  Xato {col}: {str(e)}")
            continue
    
    print(f"   ✓ {len([f for f in new_features if any(x in f for x in ['squared', 'cubed', 'sqrt', 'log'])])} polynomial features")
    

    print("\n🔨 2. Interaction features (faqat numerical)...")
    
    if len(numerical_cols) >= 2:
        for i in range(min(3, len(numerical_cols))):
            for j in range(i+1, min(4, len(numerical_cols))):
                col1, col2 = numerical_cols[i], numerical_cols[j]
                
                try:
                    # Multiplication
                    df_enh[f'{col1}_x_{col2}'] = df_enh[col1] * df_enh[col2]
                    new_features.append(f'{col1}_x_{col2}')
                    
                    # Division (avoid division by zero)
                    if (df_enh[col2] != 0).all() and df_enh[col2].std() > 0:
                        df_enh[f'{col1}_div_{col2}'] = df_enh[col1] / (df_enh[col2] + 1e-5)
                        new_features.append(f'{col1}_div_{col2}')
                    
                    # Addition
                    df_enh[f'{col1}_plus_{col2}'] = df_enh[col1] + df_enh[col2]
                    new_features.append(f'{col1}_plus_{col2}')
                    
                    # Subtraction (absolute)
                    df_enh[f'{col1}_minus_{col2}_abs'] = np.abs(df_enh[col1] - df_enh[col2])
                    new_features.append(f'{col1}_minus_{col2}_abs')
                    
                except Exception as e:
                    print(f"   ⚠️  Xato {col1} x {col2}: {str(e)}")
                    continue
    
    print(f"   ✓ {len([f for f in new_features if any(x in f for x in ['_x_', '_div_', '_plus_', '_minus_'])])} interaction features")
    
    print("\n🔨 3. Ratio features (faqat numerical)...")
    
    if len(numerical_cols) >= 2:
        for i, col1 in enumerate(numerical_cols[:3]):
            for col2 in numerical_cols[i+1:4]:
                try:
                    if (df_enh[col1] + df_enh[col2]).std() > 0:
                        df_enh[f'{col1}_to_{col2}_ratio'] = (
                            df_enh[col1] / (df_enh[col1] + df_enh[col2] + 1e-5)
                        )
                        new_features.append(f'{col1}_to_{col2}_ratio')
                except Exception as e:
                    continue
    
    print(f"   ✓ {len([f for f in new_features if '_ratio' in f])} ratio features")
    

    print("\n🔨 4. Categorical aggregations...")
    
    # Har bir categorical feature uchun numerical features'ning statistikasi
    for cat_col in categorical_cols[:min(3, len(categorical_cols))]:
        for num_col in numerical_cols[:min(3, len(numerical_cols))]:
            try:
                # Mean encoding
                mean_encoded = df_enh.groupby(cat_col)[num_col].transform('mean')
                df_enh[f'{cat_col}_{num_col}_mean'] = mean_encoded
                new_features.append(f'{cat_col}_{num_col}_mean')
                
                # Std encoding
                std_encoded = df_enh.groupby(cat_col)[num_col].transform('std').fillna(0)
                df_enh[f'{cat_col}_{num_col}_std'] = std_encoded
                new_features.append(f'{cat_col}_{num_col}_std')
                
                # Count encoding
                count_encoded = df_enh.groupby(cat_col)[num_col].transform('count')
                df_enh[f'{cat_col}_{num_col}_count'] = count_encoded
                new_features.append(f'{cat_col}_{num_col}_count')
                
            except Exception as e:
                print(f"   ⚠️  Xato {cat_col} x {num_col}: {str(e)}")
                continue
    
    print(f"   ✓ {len([f for f in new_features if any(x in f for x in ['_mean', '_std', '_count'])])} categorical aggregation features")
    

    print("\n🔨 5. Frequency encoding (categorical)...")
    
    for cat_col in categorical_cols[:min(5, len(categorical_cols))]:
        try:
            freq = df_enh[cat_col].value_counts(normalize=True)
            df_enh[f'{cat_col}_freq'] = df_enh[cat_col].map(freq)
            new_features.append(f'{cat_col}_freq')
        except Exception as e:
            continue
    
    print(f"   ✓ {len([f for f in new_features if '_freq' in f])} frequency features")
    

    print("\n🔨 6. Statistical aggregation...")
    
    if len(numerical_cols) >= 3:
        selected_num_cols = numerical_cols[:min(10, len(numerical_cols))]
        
        try:
            df_enh['row_mean'] = df_enh[selected_num_cols].mean(axis=1)
            df_enh['row_std'] = df_enh[selected_num_cols].std(axis=1)
            df_enh['row_max'] = df_enh[selected_num_cols].max(axis=1)
            df_enh['row_min'] = df_enh[selected_num_cols].min(axis=1)
            df_enh['row_median'] = df_enh[selected_num_cols].median(axis=1)
            df_enh['row_range'] = df_enh['row_max'] - df_enh['row_min']
            
            new_features.extend(['row_mean', 'row_std', 'row_max', 'row_min', 'row_median', 'row_range'])
            print(f"   ✓ 6 statistical features")
        except Exception as e:
            print(f"   ⚠️  Statistical aggregation xato: {str(e)}")
    
    print("\n🔨 7. Binning features (numerical)...")
    
    for col in numerical_cols[:min(5, len(numerical_cols))]:
        try:
            df_enh[f'{col}_bin'] = pd.qcut(
                df_enh[col], q=5, labels=False, duplicates='drop'
            )
            new_features.append(f'{col}_bin')
        except Exception as e:
            # Agar qcut ishlamasa, cut ishlatamiz
            try:
                df_enh[f'{col}_bin'] = pd.cut(
                    df_enh[col], bins=5, labels=False, duplicates='drop'
                )
                new_features.append(f'{col}_bin')
            except:
                continue
    
    print(f"   ✓ {len([f for f in new_features if '_bin' in f])} binning features")
    
    print("\n🔨 8. Missing value indicators...")
    
    missing_cols = df_enh.columns[df_enh.isnull().any()].tolist()
    for col in missing_cols:
        df_enh[f'{col}_is_missing'] = df_enh[col].isnull().astype(int)
        new_features.append(f'{col}_is_missing')
    
    print(f"   ✓ {len(missing_cols)} missing indicators")
    print("\n🔧 Filling missing values...")
    
    # Numerical columns - median bilan
    num_cols_in_df = df_enh.select_dtypes(include=[np.number]).columns
    df_enh[num_cols_in_df] = df_enh[num_cols_in_df].fillna(df_enh[num_cols_in_df].median())
    
    # Categorical columns - mode bilan (agar bo'lsa)
    cat_cols_in_df = df_enh.select_dtypes(exclude=[np.number]).columns
    for col in cat_cols_in_df:
        df_enh[col] = df_enh[col].fillna(df_enh[col].mode()[0] if len(df_enh[col].mode()) > 0 else 0)
    
    print(f"   ✓ Missing values filled")
    print(f"\n{'='*70}")
    print(f"✅ FEATURE ENGINEERING YAKUNLANDI!")
    print(f"{'='*70}")
    print(f"   Original features:     {df.shape[1]}")
    print(f"   New features created:  {len(new_features)}")
    print(f"   Total features:        {df_enh.shape[1]}")
    print(f"   - Numerical features:  {len(df_enh.select_dtypes(include=[np.number]).columns)}")
    print(f"   - Non-numerical:       {len(df_enh.select_dtypes(exclude=[np.number]).columns)}")
    print(f"{'='*70}")
    
    return df_enh, new_features, feature_types
print("🔨 Feature engineering...")
feature_types = identify_feature_types(X)
print(feature_types)
X_engineered, new_features, feature_types = engineer_features(X)

X_engineered=X
print(f"✓ Original features: {X.shape[1]}")
print(f"✓ Engineered features: {X_engineered.shape[1]}")

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X_engineered, y, test_size=0.2, stratify=y, random_state=42
)

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

print(f"✓ Train: {X_train_split.shape}")
print(f"✓ Val:   {X_val.shape}")
print(f"✓ Test:  {X_test.shape}")



print("\n" + "="*80)
print("STEP 2: FEATURE SELECTION")
print("="*80)

# Select top K features
k_best = min(100, X_train_split.shape[1])
print(f"🎯 Selecting top {k_best} features...")

selector = SelectKBest(score_func=mutual_info_classif, k=k_best)
selector.fit(X_train_split, y_train_split)

X_train_selected = pd.DataFrame(
    selector.transform(X_train_split),
    columns=selector.get_feature_names_out(),
    index=X_train_split.index
)
X_val_selected = pd.DataFrame(
    selector.transform(X_val),
    columns=selector.get_feature_names_out(),
    index=X_val.index
)
X_test_selected = pd.DataFrame(
    selector.transform(X_test),
    columns=selector.get_feature_names_out(),
    index=X_test.index
)

print(f"✓ Selected features: {X_train_selected.shape[1]}")

scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_selected),
    columns=X_train_selected.columns,
    index=X_train_selected.index
)
X_val_scaled = pd.DataFrame(
    scaler.transform(X_val_selected),
    columns=X_val_selected.columns,
    index=X_val_selected.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_selected),
    columns=X_test_selected.columns,
    index=X_test_selected.index
)

print("✓ Scaling completed!")
print("\n" + "="*80)
print("STEP 4: TRAINING ENSEMBLE MODELS")
print("="*80)

# Model 1: CatBoost (original data)
print("\n1. CatBoost (tuned)...")
model_cb = CatBoostClassifier(
    iterations=700,
    learning_rate=0.03,
    depth=9,
    l2_leaf_reg=5,
    class_weights=[1, 20],
    random_seed=42,
    verbose=False
)
model_cb.fit(X_train_scaled, y_train_split)
print("   ✓ Trained")

# Model 2: LightGBM (original data)
print("\n2. LightGBM (tuned)...")
model_lgbm = LGBMClassifier(
    n_estimators=700,
    learning_rate=0.03,
    max_depth=9,
    num_leaves=50,
    class_weight={0: 1, 1: 20},
    random_state=42,
    verbose=-1
)
model_lgbm.fit(X_train_scaled, y_train_split)
print("   ✓ Trained")

# Model 3: XGBoost (original data)
print("\n3. XGBoost (tuned)...")
model_xgb = XGBClassifier(
    n_estimators=700,
    learning_rate=0.03,
    max_depth=9,
    scale_pos_weight=20,
    random_state=42,
    verbosity=0
)
model_xgb.fit(X_train_scaled, y_train_split)
print("   ✓ Trained")

# Model 4: CatBoost with SMOTE
print("\n4. CatBoost + SMOTE...")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train_split)
model_smote = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    random_seed=42,
    verbose=False
)
model_smote.fit(X_train_smote, y_train_smote)
print("   ✓ Trained")

# Model 5: CatBoost with ADASYN
print("\n5. CatBoost + ADASYN...")
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_scaled, y_train_split)
model_adasyn = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    random_seed=42,
    verbose=False
)
model_adasyn.fit(X_train_adasyn, y_train_adasyn)
print("   ✓ Trained")

# Model 6: Calibrated CatBoost
print("\n6. Calibrated CatBoost...")
model_calibrated = CalibratedClassifierCV(model_cb, method='isotonic', cv=5)
model_calibrated.fit(X_train_scaled, y_train_split)
print("   ✓ Trained")

print("\n✅ All 6 models trained successfully!")


🔨 Feature engineering...
{'numerical': ['loan_amount', 'interest_rate', 'loan_to_value_ratio', 'age', 'annual_income', 'employment_length', 'regional_unemployment_rate', 'regional_median_income', 'regional_median_rent', 'housing_price_index', 'cost_of_living_index', 'num_credit_accounts', 'oldest_credit_line_age', 'oldest_account_age_months', 'total_credit_limit', 'account_diversity_index', 'application_hour', 'num_login_sessions', 'num_customer_service_calls', 'account_tenure', 'monthly_income', 'existing_monthly_debt', 'monthly_payment', 'debt_service_ratio', 'payment_to_income_ratio', 'credit_utilization', 'credit_usage_amount', 'available_credit', 'total_monthly_debt_payment', 'annual_debt_payment', 'loan_to_annual_income', 'total_debt_amount', 'monthly_free_cash_flow'], 'categorical': ['loan_type', 'loan_term', 'loan_purpose', 'origination_channel', 'employment_type', 'education', 'marital_status', 'num_dependents', 'num_delinquencies_2yrs', 'num_inquiries_6mo', 'recent_inquiry_co

In [ ]:
print("\n" + "="*80)
print("STEP 5: ENSEMBLE PREDICTION")
print("="*80)

# Get predictions from all models
predictions_val = [
    model_cb.predict_proba(X_val_scaled)[:, 1],
    model_lgbm.predict_proba(X_val_scaled)[:, 1],
    model_xgb.predict_proba(X_val_scaled)[:, 1],
    model_smote.predict_proba(X_val_scaled)[:, 1],
    model_adasyn.predict_proba(X_val_scaled)[:, 1],
    model_calibrated.predict_proba(X_val_scaled)[:, 1]
]

predictions_test = [
    model_cb.predict_proba(X_test_scaled)[:, 1],
    model_lgbm.predict_proba(X_test_scaled)[:, 1],
    model_xgb.predict_proba(X_test_scaled)[:, 1],
    model_smote.predict_proba(X_test_scaled)[:, 1],
    model_adasyn.predict_proba(X_test_scaled)[:, 1],
    model_calibrated.predict_proba(X_test_scaled)[:, 1]
]
# Optimize ensemble weights
from scipy.optimize import minimize
from sklearn.metrics import roc_auc_score

def ensemble_objective(weights):
    y_proba = np.average(predictions_val, axis=0, weights=weights)
    return -roc_auc_score(y_val, y_proba)

constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
bounds = [(0, 1) for _ in range(6)]
initial_weights = [1/6] * 6

result = minimize(ensemble_objective, initial_weights, bounds=bounds, constraints=constraints)
optimal_weights = result.x

print("✓ Optimal ensemble weights:")
model_names = ['CatBoost', 'LightGBM', 'XGBoost', 'SMOTE', 'ADASYN', 'Calibrated']
for name, weight in zip(model_names, optimal_weights):
    print(f"   {name:15s}: {weight:.3f}")

# Final ensemble predictions
y_proba_val = np.average(predictions_val, axis=0, weights=optimal_weights)
y_proba_test = np.average(predictions_test, axis=0, weights=optimal_weights)

print("\n" + "="*80)
print("STEP 6: OPTIMAL THRESHOLD")
print("="*80)

from sklearn.metrics import confusion_matrix, average_precision_score

thresholds = np.arange(0.1, 0.9, 0.01)
costs = []
for threshold in thresholds:
    y_pred = (y_proba_val >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    total_cost = (fn * 10000) + (fp * 500)
    costs.append(total_cost)

optimal_threshold = thresholds[np.argmin(costs)]
print(f"✓ Optimal threshold: {optimal_threshold:.3f}")

print("\n" + "="*80)
print("STEP 7: TEST SET EVALUATION")
print("="*80)

roc_auc_test = roc_auc_score(y_test, y_proba_test)
pr_auc_test = average_precision_score(y_test, y_proba_test)

y_pred_test = (y_proba_test >= optimal_threshold).astype(int)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
total_cost = (fn * 10000) + (fp * 500)

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"\n📊 FINAL TEST METRICS:")
print(f"   ROC-AUC:     {roc_auc_test:.4f}")
print(f"   PR-AUC:      {pr_auc_test:.4f}")
print(f"   Precision:   {precision:.3f}")
print(f"   Recall:      {recall:.3f}")
print(f"   F1-Score:    {f1:.3f}")
print(f"   Threshold:   {optimal_threshold:.3f}")
print(f"   False Neg:   {fn}")
print(f"   Total Cost:  ${total_cost:,}")



STEP 5: ENSEMBLE PREDICTION
✓ Optimal ensemble weights:
   CatBoost       : 0.167
   LightGBM       : 0.167
   XGBoost        : 0.167
   SMOTE          : 0.167
   ADASYN         : 0.167
   Calibrated     : 0.167

STEP 6: OPTIMAL THRESHOLD
✓ Optimal threshold: 0.110

STEP 7: TEST SET EVALUATION

📊 FINAL TEST METRICS:
   ROC-AUC:     0.7850
   PR-AUC:      0.2174
   Precision:   0.108
   Recall:      0.749
   F1-Score:    0.188
   Threshold:   0.110
   False Neg:   231
   Total Cost:  $5,165,000


: 